In [1]:
!pip install torch torchvision
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install ultralytics
!pip install fiftyone opencv-python-headless

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-5zyy8d86
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-5zyy8d86
  Resolved https://github.com/facebookresearch/detectron2.git to commit ebe8b45437f86395352ab13402ba45b75b4d1ddb
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.6 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 51.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.9 MB/s eta 0:00:

In [4]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os
import cv2
import fiftyone as fo
import fiftyone.zoo as foz
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=3000,
    shuffle=True
)# Load 3000 samples from COCO


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))# Set up Detectron2 configuration
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


def runInference(image_path):# Function to run inference on an image
    im= cv2.imread(image_path)
    outputs = predictor(im)
    return outputs, im


for sample in dataset.iter_samples():# Run inference and add predictions to the dataset
    image_path = sample.filepath
    outputs, im = runInference(image_path)


    detections = []
    for i in range(len(outputs["instances"])):
        bbox = outputs["instances"].pred_boxes[i].tensor.cpu().numpy()[0]
        score = outputs["instances"].scores[i].cpu().item()
        label = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[outputs["instances"].pred_classes[i]]

        detection = fo.Detection(
            label=label,
            bounding_box=[
                bbox[0] / im.shape[1],
                bbox[1] / im.shape[0],
                (bbox[2] - bbox[0]) / im.shape[1],
                (bbox[3] - bbox[1]) / im.shape[0]
            ],
            confidence=score
        )
        detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions


print("\nDetailed Evaluation Report:")
print(results.report())

# From the results we can get the weightes average

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Sufficient images already downloaded


INFO:fiftyone.utils.coco:Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 100/100 [1.5s elapsed, 0s remaining, 68.9 samples/s]         


INFO:eta.core.utils: 100% |█████████████████| 100/100 [1.5s elapsed, 0s remaining, 68.9 samples/s]         



Detailed Evaluation Report:
{'apple': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, 'backpack': {'precision': 0.375, 'recall': 0.3, 'f1-score': 0.33333333333333326, 'support': 10.0}, 'banana': {'precision': 0.8, 'recall': 1.0, 'f1-score': 0.888888888888889, 'support': 4.0}, 'baseball bat': {'precision': 1.0, 'recall': 0.6, 'f1-score': 0.7499999999999999, 'support': 5.0}, 'baseball glove': {'precision': 0.8, 'recall': 0.6666666666666666, 'f1-score': 0.7272727272727272, 'support': 6.0}, 'bear': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3.0}, 'bed': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 3.0}, 'bench': {'precision': 0.2, 'recall': 0.2, 'f1-score': 0.20000000000000004, 'support': 5.0}, 'bicycle': {'precision': 0.5833333333333334, 'recall': 0.5, 'f1-score': 0.5384615384615384, 'support': 14.0}, 'bird': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1.0}, 'boat': {'precision': 0.3333333333333333,

In [ ]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os
import cv2
import fiftyone as fo
import fiftyone.zoo as foz
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=1000,
    shuffle=True
)# Load 1000 samples from COCO dataset


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))# Set up Detectron2 configuration for Faster R-CNN with ResNet-101 backbone
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


def runInference(image_path):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    return outputs, im

# Run inference and add predictions to the dataset
for sample in dataset.iter_samples():
    image_path = sample.filepath
    outputs, im = runInference(image_path)

    # Convert Detectron2 outputs to FiftyOne format
    detections = []
    for i in range(len(outputs["instances"])):
        bbox = outputs["instances"].pred_boxes[i].tensor.cpu().numpy()[0]
        score = outputs["instances"].scores[i].cpu().item()
        label = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[outputs["instances"].pred_classes[i]]

        detection = fo.Detection(
            label=label,
            bounding_box=[
                bbox[0] / im.shape[1],
                bbox[1] / im.shape[0],
                (bbox[2] - bbox[0]) / im.shape[1],
                (bbox[3] - bbox[1]) / im.shape[0]
            ],
            confidence=score
        )
        detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions

print("\nDetailed Evaluation Report:")
print(results.report())


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 149 (<3000) samples matching your requirements


Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 149/149 [4.5s elapsed, 0s remaining, 36.9 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 149/149 [4.5s elapsed, 0s remaining, 36.9 samples/s]       



Detailed Evaluation Report:
{'airplane': {'precision': 0.8, 'recall': 0.8, 'f1-score': 0.8000000000000002, 'support': 5}, 'apple': {'precision': 0.3333333333333333, 'recall': 0.5, 'f1-score': 0.4, 'support': 6}, 'backpack': {'precision': 0.5517241379310345, 'recall': 0.2909090909090909, 'f1-score': 0.38095238095238093, 'support': 55}, 'banana': {'precision': 0.9, 'recall': 0.4090909090909091, 'f1-score': 0.5625000000000001, 'support': 22}, 'baseball bat': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'bed': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'bench': {'precision': 0.55, 'recall': 0.3793103448275862, 'f1-score': 0.4489795918367347, 'support': 29}, 'bicycle': {'precision': 0.7415254237288136, 'recall': 0.54858934169279, 'f1-score': 0.6306306306306306, 'support': 319}, 'bird': {'precision': 1.0, 'recall': 0.2, 'f1-score': 0.33333333333333337, 'support': 15}, 'boat': {'precision': 0.7, 'recall': 0.65625, 'f1-score': 0.6774193548387096, 's

In [ ]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import fiftyone as fo
import fiftyone.zoo as foz
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=1000,
    shuffle=True
)# Load 1000 samples from COCO dataset


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))# Set up Detectron2 configuration for Mask R-CNN
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


def runInference(image_path):# Function to run inference on an image
    im = cv2.imread(image_path)
    outputs = predictor(im)
    return outputs, im


for sample in dataset.iter_samples():
    image_path = sample.filepath
    outputs, im = runInference(image_path)


    detections = []
    instances = outputs["instances"].to("cpu")
    for i in range(len(instances)):
        bbox = instances.pred_boxes[i].tensor.numpy()[0]
        score = instances.scores[i].item()
        label = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[instances.pred_classes[i].item()]

        # If masks are present, extract and process them
        mask = instances.pred_masks[i].numpy().astype(np.uint8) * 255
        mask = cv2.resize(mask, (im.shape[1], im.shape[0]))  # Resize mask to match image dimensions

        detection = fo.Detection(
            label=label,
            bounding_box=[
                bbox[0] / im.shape[1],
                bbox[1] / im.shape[0],
                (bbox[2] - bbox[0]) / im.shape[1],
                (bbox[3] - bbox[1]) / im.shape[0]
            ],
            confidence=score,
            mask=mask
        )
        detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions


print("\nDetailed Evaluation Report:")
print(results.report())




INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 149 (<3000) samples matching your requirements


Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 149/149 [6.4s elapsed, 0s remaining, 22.7 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 149/149 [6.4s elapsed, 0s remaining, 22.7 samples/s]       


Mean Average Precision (mAP):
0.2963747632673844

Detailed Evaluation Report:
{'airplane': {'precision': 0.5, 'recall': 0.8, 'f1-score': 0.6153846153846154, 'support': 5}, 'apple': {'precision': 0.6666666666666666, 'recall': 0.3333333333333333, 'f1-score': 0.4444444444444444, 'support': 6}, 'backpack': {'precision': 0.6956521739130435, 'recall': 0.2909090909090909, 'f1-score': 0.41025641025641024, 'support': 55}, 'banana': {'precision': 1.0, 'recall': 0.2857142857142857, 'f1-score': 0.4444444444444445, 'support': 21}, 'bed': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'bench': {'precision': 0.5789473684210527, 'recall': 0.3793103448275862, 'f1-score': 0.45833333333333337, 'support': 29}, 'bicycle': {'precision': 0.7188755020080321, 'recall': 0.5628930817610063, 'f1-score': 0.6313932980599647, 'support': 318}, 'bird': {'precision': 0.75, 'recall': 0.2, 'f1-score': 0.31578947368421056, 'support': 15}, 'boat': {'precision': 0.75, 'recall': 0.65625, 'f1-score': 0.7, '

In [ ]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import fiftyone as fo
import fiftyone.zoo as foz
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=3000,
    shuffle=True
)# Load 3000 samples from COCO dataset


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))# Set up Detectron2 configuration for Mask R-CNN with ResNet-101 backbone
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


def runInference(image_path):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    return outputs, im

# Run inference and add predictions to the dataset
for sample in dataset.iter_samples():
    image_path = sample.filepath
    outputs, im = runInference(image_path)

    # Convert Detectron2 outputs to FiftyOne format
    detections = []
    instances = outputs["instances"].to("cpu")
    for i in range(len(instances)):
        bbox = instances.pred_boxes[i].tensor.numpy()[0]
        score = instances.scores[i].item()
        label = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[instances.pred_classes[i].item()]

        # If masks are present, extract and process them
        mask = instances.pred_masks[i].numpy().astype(np.uint8) * 255
        mask = cv2.resize(mask, (im.shape[1], im.shape[0]))  # Resize mask to match image dimensions

        detection = fo.Detection(
            label=label,
            bounding_box=[
                bbox[0] / im.shape[1],
                bbox[1] / im.shape[0],
                (bbox[2] - bbox[0]) / im.shape[1],
                (bbox[3] - bbox[1]) / im.shape[0]
            ],
            confidence=score,
            mask=mask
        )
        detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions

# Print evaluation results
print("\nDetailed Evaluation Report:")
print(results.report())


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 149 (<3000) samples matching your requirements


Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 149/149 [8.1s elapsed, 0s remaining, 15.4 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 149/149 [8.1s elapsed, 0s remaining, 15.4 samples/s]       


Mean Average Precision (mAP):
0.30535134965939886

Detailed Evaluation Report:
{'airplane': {'precision': 0.6666666666666666, 'recall': 0.8, 'f1-score': 0.7272727272727272, 'support': 5}, 'apple': {'precision': 0.6666666666666666, 'recall': 0.3333333333333333, 'f1-score': 0.4444444444444444, 'support': 6}, 'backpack': {'precision': 0.6785714285714286, 'recall': 0.34545454545454546, 'f1-score': 0.45783132530120485, 'support': 55}, 'banana': {'precision': 1.0, 'recall': 0.25, 'f1-score': 0.4, 'support': 20}, 'baseball glove': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'bed': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'bench': {'precision': 0.5, 'recall': 0.3793103448275862, 'f1-score': 0.4313725490196078, 'support': 29}, 'bicycle': {'precision': 0.7448559670781894, 'recall': 0.5727848101265823, 'f1-score': 0.6475849731663686, 'support': 316}, 'bird': {'precision': 0.4, 'recall': 0.13333333333333333, 'f1-score': 0.2, 'support': 15}, 'boat': {'

In [ ]:
import torch
import cv2
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from ultralytics import YOLO


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=3000,
    shuffle=True
)


model = YOLO("yolov8l.pt") # Load YOLOv8L model

# Function to run inference on an image
def run_inference(image_path):
    im = cv2.imread(image_path)
    results = model(im)
    return results, im

# Map class indices to class names
coco_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
               'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
               'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
               'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
               'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
               'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
               'hair drier', 'toothbrush']

class_to_id = {cls: i for i, cls in enumerate(coco_classes)}
id_to_class = {v: k for k, v in class_to_id.items()}


for sample in dataset.iter_samples():
    image_path = sample.filepath
    results, im = run_inference(image_path)

    # Convert YOLO outputs to FiftyOne format
    detections = []
    for result in results:
        # Ensure results are in the expected format
        if hasattr(result, 'boxes') and result.boxes is not None:
            for bbox, score, class_id in zip(result.boxes.xyxy.tolist(), result.boxes.conf.tolist(), result.boxes.cls.tolist()):
                bbox = bbox  # Bounding box format is already a list of floats
                score = float(score)
                class_id = int(class_id)

                # Check if class_id is within the range of class_to_id
                if class_id not in class_to_id.values():
                    continue  # Skip this detection

                label = id_to_class[class_id]  # Convert class_id to string label

                detection = fo.Detection(
                    label=label,
                    bounding_box=[
                        bbox[0] / im.shape[1],
                        bbox[1] / im.shape[0],
                        (bbox[2] - bbox[0]) / im.shape[1],
                        (bbox[3] - bbox[1]) / im.shape[0]
                    ],
                    confidence=score
                )
                detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions

print("\nDetailed Evaluation Report:")
print(results.report())

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 149 (<3000) samples matching your requirements


Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 149/149 [5.5s elapsed, 0s remaining, 35.5 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 149/149 [5.5s elapsed, 0s remaining, 35.5 samples/s]       


Mean Average Precision (mAP):
0.3540234344650885

Detailed Evaluation Report:
{'airplane': {'precision': 1.0, 'recall': 0.6, 'f1-score': 0.7499999999999999, 'support': 5}, 'apple': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 6}, 'backpack': {'precision': 0.5483870967741935, 'recall': 0.3090909090909091, 'f1-score': 0.39534883720930236, 'support': 55}, 'banana': {'precision': 1.0, 'recall': 0.3684210526315789, 'f1-score': 0.5384615384615384, 'support': 19}, 'bed': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1}, 'bench': {'precision': 0.5, 'recall': 0.5862068965517241, 'f1-score': 0.5396825396825397, 'support': 29}, 'bicycle': {'precision': 0.7751004016064257, 'recall': 0.6107594936708861, 'f1-score': 0.6831858407079646, 'support': 316}, 'bird': {'precision': 1.0, 'recall': 0.2, 'f1-score': 0.33333333333333337, 'support': 15}, 'boat': {'precision': 0.9230769230769231, 'recall': 0.7272727272727273, 'f1-score'

In [ ]:
import torch
import cv2
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from ultralytics import YOLO

# Load 10 samples from COCO dataset with 2 specific classes
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=3000,
    shuffle=True
)


model = YOLO("yolov8x.pt")  # # Load YOLOv8X model

# Function to run inference on an image
def run_inference(image_path):
    im = cv2.imread(image_path)
    results = model(im)
    return results, im

# Map class indices to class names
coco_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
               'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
               'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
               'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
               'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
               'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
               'hair drier', 'toothbrush']

class_to_id = {cls: i for i, cls in enumerate(coco_classes)}
id_to_class = {v: k for k, v in class_to_id.items()}


for sample in dataset.iter_samples():
    image_path = sample.filepath
    results, im = run_inference(image_path)

    # Convert YOLO outputs to FiftyOne format
    detections = []
    for result in results:
        # Ensure results are in the expected format
        if hasattr(result, 'boxes') and result.boxes is not None:
            for bbox, score, class_id in zip(result.boxes.xyxy.tolist(), result.boxes.conf.tolist(), result.boxes.cls.tolist()):
                bbox = bbox  # Bounding box format is already a list of floats
                score = float(score)
                class_id = int(class_id)

                # Check if class_id is within the range of class_to_id
                if class_id not in class_to_id.values():
                    continue  # Skip this detection

                label = id_to_class[class_id]  # Convert class_id to string label

                detection = fo.Detection(
                    label=label,
                    bounding_box=[
                        bbox[0] / im.shape[1],
                        bbox[1] / im.shape[0],
                        (bbox[2] - bbox[0]) / im.shape[1],
                        (bbox[3] - bbox[1]) / im.shape[0]
                    ],
                    confidence=score
                )
                detections.append(detection)

    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()


results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True
)# Evaluate the predictions


print("\nDetailed Evaluation Report:")
print(results.report())

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 149 (<3000) samples matching your requirements


Sufficient images already downloaded


...

Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 149/149 [7.2s elapsed, 0s remaining, 25.3 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 149/149 [7.2s elapsed, 0s remaining, 25.3 samples/s]       


Mean Average Precision (mAP):
0.3866536614882825

Detailed Evaluation Report:
{'airplane': {'precision': 0.8, 'recall': 0.8, 'f1-score': 0.8000000000000002, 'support': 5}, 'apple': {'precision': 0.2857142857142857, 'recall': 0.3333333333333333, 'f1-score': 0.30769230769230765, 'support': 6}, 'backpack': {'precision': 0.6363636363636364, 'recall': 0.38181818181818183, 'f1-score': 0.47727272727272735, 'support': 55}, 'banana': {'precision': 0.75, 'recall': 0.3157894736842105, 'f1-score': 0.44444444444444436, 'support': 19}, 'bed': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'bench': {'precision': 0.6060606060606061, 'recall': 0.6896551724137931, 'f1-score': 0.6451612903225807, 'support': 29}, 'bicycle': {'precision': 0.7276119402985075, 'recall': 0.6151419558359621, 'f1-score': 0.6666666666666666, 'support': 317}, 'bird': {'precision': 1.0, 'recall': 0.2, 'f1-score': 0.33333333333333337, 'support': 15}, 'boat': {'precision': 0.8518518518518519, 'recall': 0.71875, 'f